In [34]:
import numpy as np
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup as bs
from IPython.display import clear_output, display

In [35]:
### ALL FUNCTIONS HERE ###

In [36]:
def getBookMeta(isbn):
    api_url = 'https://www.googleapis.com/books/v1/volumes?q=isbn:'
    isbn_number = isbn
    url = api_url + str(isbn_number)
    return json.loads(requests.get(url).content)

In [37]:
apiRestBooks = []
def apiRest(isbn):
  apiRestBooks.append(isbn)

In [38]:
## check is the book with ISBN has data on Goodreads return True/False ##
def GRhasBook(isbn):
  url = 'https://www.goodreads.com/book/review_counts.json?isbns='
  result = requests.get(url+str(isbn)).text
  if result == 'No books match those ISBNs.':
    return False
  elif result == 'No ISBNs specified.':
    return False
  else:
    return True

In [39]:
def getGRinit(isbn):
  if GRhasBook(isbn):
    url = 'https://www.goodreads.com/book/review_counts.json?isbns='
    result = json.loads(requests.get(url+str(isbn)).content)
    return result.get('books')
  else:
    print('NO BOOKS FOUND ... ', isbn)

In [40]:
def GRhasRatings(isbn):
  stat = getGRreviewStat(isbn)[0]
  ratings_cnt = stat[2]
  reviews_cnt = stat[3]
  if ratings_cnt == 0 or reviews_cnt == 0:
    return False
  else:
    return True

In [41]:
## return list of reviews stat of the book ##
def getGRreviewStat(isbn):
  bookStats = []
  url = 'https://www.goodreads.com/book/review_counts.json?isbns='
  if GRhasBook(isbn):
    meta = json.loads(requests.get(url+str(isbn)).text).get('books')[0]
    ratings_cnt = meta.get('work_ratings_count')
    text_reviews_cnt = meta.get('work_text_reviews_count')
    reviews_cnt = meta.get('work_reviews_count')
    avg_rating = meta.get('average_rating')
    bookStats.append([avg_rating, reviews_cnt, ratings_cnt, text_reviews_cnt,])
    return bookStats
  else:
    return [['','','','','']]

In [42]:
noBooksList = []
noStatList = []
## return list of book attributes ##
def getGRmeta(isbn):
  if GRhasBook(isbn):
    url = 'https://www.goodreads.com/book/isbn/'+str(isbn)
    page = requests.get(url)
    global soup_author
    soup_author = bs(page.content, 'html.parser')

    #if not not404:
    #  return

    ### /!\ ATTENTION /!\ ###
    ### loop for author name due to some kind of weird url request or dynamic content thing can lead to non auther =) ###
    ### take so long time but can get complete data used when GoogleBooks API not found the books ONLY ###
    while len(soup_author.find_all('a', class_ = 'authorName')) < 1:
      url = 'https://www.goodreads.com/book/isbn/'+str(isbn)
      page = requests.get(url)
      soup_author = bs(page.content, 'html.parser')

    bookAttributes = []
    author_list = []
    for author in soup_author.find_all('a', class_ = 'authorName'):
      author_list.append(author.string)
    bookTitle = soup_author.find_all('title')[0].string

    ### book statistics process ###
    bookStat = getGRreviewStat(isbn)[0] # list of book stat [[average_rating, #reviews, #ratings, #text_reviews]]
    rating_avg = bookStat[0]
    reviews_cnt = bookStat[1]
    ratings_cnt = bookStat[2]
    text_reviews_cnt = bookStat[3]
    ### end of statistic process ###

    ### Append all attributes ###
    bookAttributes.append([author_list, bookTitle])
    return bookAttributes
  else:
    print('no book: ', isbn)

In [43]:
def getID():
  id = len(dfBooks) + 1
  return id

In [44]:
def addRow(meta):
    adding_book = pd.DataFrame(meta, columns=['id', 'isbn_10', 'isbn_13', 'isbn_other', 'isbn_book', 'authors', 'title', 'subtitle', 'publisher', 'published_date', 'page_count', 'categories', 'language', 'google_desc', 'rating_avg', '#reviews', '#ratings', '#text_reviews', 'thumbnail', 'genre', 'goodreads_desc', 'text_reviews', 'also_enjoy', 'status'])
    global dfBooks
    dfBooks = pd.concat([dfBooks, adding_book], ignore_index = True)

In [45]:
def addBook(isbn):
    book_meta = getBookMeta(isbn)
    total_items = book_meta['totalItems']
    if total_items == 0:
      global noBooksNum
      noBooksList.append(isbn)
      #noBooksNum = noBooksNum + 1
      clear_output(wait=True)
      display(len(noBooksList),' no book found: ', isbn)
      return

    book_meta_item = book_meta.get('items')
    meta = []

    ## attributes ##
    #try:
    for item in range(len(book_meta_item)):
      metaData = book_meta_item[item].get('volumeInfo')
      #google_id = book_meta_item[item]['id']

      ### isbn process ###
      if GRhasBook(isbn):
        GRini = getGRinit(isbn)[0]
        # initialize isbn #
        isbn_10 = GRini.get('isbn')
        isbn_13 = str(GRini.get('isbn13'))
        isbn_other = None
        isbn_book = isbn
      else:
        isbn_10 = None
        isbn_13 = None
        isbn_other = None
        isbn_book = isbn
      ### end of isbn process ###

      authors = metaData.get('authors','')
      title = metaData.get('title','')
      subtitle = metaData.get('subtitle','')
      publisher = metaData.get('publisher','')
      published_date = metaData.get('publishedDate','')
      page_count = metaData.get('pageCount','')
      categories = metaData.get('categories','')
      language = metaData.get('language','')
      google_desc = metaData.get('description','')
      #rating_avg = metaData.get('averageRating','')
      #rating_count = metaData.get('ratingsCount','')

      ### thumbnail process ###
      global img_url
      if metaData.get('imageLinks','') is '':
        img_url = ''
      else:
        img_url = metaData.get('imageLinks','').get('thumbnail','')
      ### end thumbnail process ###


      ### check if the book has ISBN_13 (ONLY ISBN_13 WILL GET STAT) ###
      if isbn_13 is None:
        noStatList.append(isbn)
        #global books_cnt
        #books_cnt = books_cnt + 1
        global dfBooks
        id = getID()
        meta.append([id, isbn_10, isbn_13, isbn_other, isbn_book, authors, title, subtitle, publisher, published_date, page_count, categories, language, google_desc, None, None, None, None, img_url, None, None, None, None, -1])
        addRow(meta)
        meta = []
        continue

      ### book statistics process ###
      bookStat = getGRreviewStat(isbn_13)[0] # list of book stat [[average_rating, #reviews, #ratings, #text_reviews]]
      rating_avg = bookStat[0]
      reviews_cnt = bookStat[1]
      ratings_cnt = bookStat[2]
      text_reviews_cnt = bookStat[3]
      ### end of statistic process ###

      #global books_cnt
      #books_cnt = books_cnt + 1
      id = getID()
      meta.append([id, isbn_10, isbn_13, isbn_other, isbn_book, authors, title, subtitle, publisher, published_date, page_count, categories, language, google_desc, rating_avg, reviews_cnt, ratings_cnt, text_reviews_cnt, img_url, None, None, None, None, -1])

      addRow(meta)
      meta = []
      
    #adding_book = pd.DataFrame(meta, columns=['id', 'isbn_10', 'isbn_13', 'isbn_other', 'isbn_book', 'authors', 'title', 'subtitle', 'publisher', 'published_date', 'page_count', 'categories', 'language', 'google_desc', 'rating_avg', '#reviews', '#ratings', '#text_reviews', 'thumbnail'])
    #global dfBooks
    #dfBooks = pd.concat([dfBooks, adding_book], ignore_index = True)


<>:50: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:50: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-45-ca139abc6c77>:50: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if metaData.get('imageLinks','') is '':


In [46]:
jsonError = []
def newBooks(books_isbn):
  for index in range(len(books_isbn)):
    isbn = books_isbn.iloc[index]['Barcode']
    if isDuplicate(isbn):
      print('Aready has book: ' + isbn)
      continue
    try:
      ### PROGRESS ###
      clear_output(wait=True)
      print('ADDING ... ',isbn , '[', index,'/',len(books_isbn),']')
      print('NO BOOKS FOUND: ', len(noBooksList), 'BOOKS')
      print('API OUT: ', len(apiRestBooks))
      print('JSON ERROR: ', len(jsonError))
      ################
      addBook(isbn)
    except KeyError as e: # key error
      if e.args[0] == 'totalItems':
        apiRestNum = apiRestNum + 1
        #clear_output(wait=True)
        display(apiRestNum, ' API OUT')
        apiRest(isbn)
    #except ValueError as v: # JSON ERROR
    #  jsonError.append(isbn)
    #  dfErr = {'Barcode':isbn}
    #  books_isbn = books_isbn.append(dfErr, ignore_index=True)
    #  display([len(jsonError),' JSON ERROR'])

In [47]:
def retryAPIout(apiRestBooks):
  tempBooks = apiRestBooks
  while len(apiRestBooks) != 0:
    for isbn in apiRestBooks:
      try:
        addBook(isbn)
        apiRestBooks.pop(apiRestBooks.index(isbn))
        apiRestNum = apiRestNum -1
        clear_output(wait=True)
        print(apiRestNum, ' LEFT')
      except KeyError as e:
        if e.args[0] == 'totalItems':
          continue
          #clear_output(wait=True)
          #display('API OUT')
      except ValueError as v: # JSON ERROR
        jsonError.append(isbn)
        dfErr = {'Barcode':isbn}
        books_isbn = books_isbn.append(dfErr, ignore_index=True)
        display([len(jsonError),' JSON ERROR'])

In [48]:
list404 = []
list500 = []
listOther = []

def not404(isbn):
  url = 'https://www.goodreads.com/book/isbn/'+str(isbn)
  page = requests.get(url)
  if page.status_code == 200:
    return True
  elif page.status_code == 404:
    list404.append(isbn)
    display('404')
    return False
  elif page.status_code == 500:
    list500.append(isbn)
    display('500')
    return False
  else:
    listOther.append(isbn)
    display('SOME HTTP ERROR')
    return False

In [49]:
def getContent(isbn):
  #clear_output(wait=True)
  print('GETTING CONTENT >>> ', isbn)
  if GRhasBook(isbn) and not404(isbn):
    url= 'https://www.goodreads.com/book/isbn/' + str(isbn)
    page = requests.get(url) 
    soup = bs(page.content, 'html.parser')
    while checkContent(soup, isbn):
      page = requests.get(url) 
      soup = bs(page.content, 'html.parser')
    return soup
  else:
    display('ISBN NOT FOUND OR HTTP ERROR')

In [50]:
def isGenre(isbn):
  acceptance_rounds = 4
  while acceptance_rounds > 0:
    url = 'https://www.goodreads.com/book/isbn/' + str(isbn)
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')
    genre = soup.find_all('a', class_ = 'actionLinkLite bookPageGenreLink')
    #clear_output(wait=True)
    if len(genre) > 0:
      display('GENRE FOUND')
      return False
      break
    else:
      acceptance_rounds = acceptance_rounds - 1
      print('NO GENRE ACCEPTANCE ROUNDS: ', acceptance_rounds)
  return True

In [51]:
def checkContent(content, isbn):
  if content is None: ## content is None will lead to AttributeError ## use 'is' instead of '=='
    return True
  ### check genre ###
  genre = content.find_all('a', class_ = 'actionLinkLite bookPageGenreLink')
  ### check text reviews ###
  textReviews = content.find_all('span', class_ = 'readable')

  ### CHECK CONDITION ###
  if len(textReviews) == 0: ## no text reviews get new request(true)
    return True
  elif len(genre) != 0 and len(textReviews) != 0: ## Good conent
    return False
  elif isGenre(isbn) and len(textReviews) != 0: ## real content have no genre and have text -> good content no need to get new request
    return False
  else: ## real content have genre but the content we get not found -> get new request(true)
    return True

In [52]:
def getGenre(content, isbn):
  genre_list = []
  if content is None:
    content = getContent(isbn)
  for index in range(len(content.find_all('a', class_ = 'actionLinkLite bookPageGenreLink'))):
    genre = content.find_all('a', class_ = 'actionLinkLite bookPageGenreLink')[index].string
    genre_list.append(genre)
  return genre_list

In [53]:
################################    #
### ADD BOOKS FROM GOODREADS ### #######
################################    #
## new initialize
def addGRbooks(noBooksList):
  noBooksNum = len(noBooksList)
  for index in range(len(noBooksList)):
    isbn = noBooksList[index]
    clear_output(wait=True)
    display(['CHECKING >>> ', isbn])
    display(['NO INFO BOOKS: ', noBooksNum, ' LEFT'])
    display(['404 REQUEST: ', len(list404)])
    if GRhasBook(isbn) and not404(isbn):
      GRini = getGRinit(isbn)[0]
      meta = getGRmeta(isbn)[0]
      ### ISBN ###
      isbn_10 = GRini.get('isbn')
      isbn_13 = str(GRini.get('isbn13'))
      ### book statistics process ###
      bookStat = getGRreviewStat(isbn)[0] # list of book stat [[average_rating, #reviews, #ratings, #text_reviews]]
      rating_avg = bookStat[0]
      reviews_cnt = bookStat[1]
      ratings_cnt = bookStat[2]
      text_reviews_cnt = bookStat[3]
      ### end of statistic process ###
      id = getID()
      attr = [[id, isbn_10, isbn_13, None, isbn, meta[0], meta[1], None, None, None, None, None, None, None, rating_avg, reviews_cnt, ratings_cnt, text_reviews_cnt, None]]
      adding_book = pd.DataFrame(attr, columns=['id', 'isbn_10', 'isbn_13', 'isbn_other', 'isbn_book', 'authors', 'title', 'subtitle', 'publisher', 'published_date', 'page_count', 'categories', 'language', 'google_desc', 'rating_avg', '#reviews', '#ratings', '#text_reviews', 'thumbnail'])
      global dfBooks
      dfBooks = pd.concat([dfBooks, adding_book], ignore_index = True)
      noBooksNum = noBooksNum - 1

In [54]:
def getStars(content, reviews_index):
  title = content.find_all('span', class_ = ' staticStars notranslate')[reviews_index].get('title')
  stars = 0
  if title == 'it was amazing':
    stars = 5.0
  elif title == 'really liked it':
    stars = 4.0
  elif title == 'liked it':
    stars = 3.0
  elif title == 'it was ok':
    stars = 2.0
  else:
    stars = 1.0
  return stars

In [55]:
def getTextReviews(content, isbn):
  textReviews_list = []
  index = 0
  if content is None:
    content = getContent(isbn)
  cnt = len(content.find_all('span', class_ = ' staticStars notranslate'))
  #clear_output(wait=True)
  print('GETTING TEXT REVIEWS >>> ', cnt, ' REVIEWS')
  for line in content.find_all('span', class_ = 'readable'):
    if cnt <= 0:
      break 
    text = line.text.splitlines()
    ### concat text reviews ###
    fullText = ''
    for each in text:
      fullText = fullText + each
    #display('GETTING STARS >>>')
    textReviews_list.append([getStars(content, index), fullText])
    index = index + 1
    cnt = cnt - 1
  return textReviews_list

In [56]:
def getGRdesc(content, isbn):
  print('GETTING DESCRIPTION ...')
  acct = 2
  cont = content.find_all('div', id = 'description')
  while len(cont) == 0 and acct != 0:
    content = getContent(isbn)
    cont = content.find_all('div', id = 'description')
    acct = acct - 1
  if len(cont) != 0:
    return cont[0].text
  else:
    return None

In [57]:
def getGRthumbnail(content, isbn):
  print('GETTING THUMBNAIL ...')
  acct = 2
  cont = content.find_all('img', id = 'coverImage')
  while len(cont) == 0 and acct != 0:
    content = getContent(isbn)
    cont = content.find_all('img', id = 'coverImage')
    acct = acct - 1
  if len(cont) != 0:
    return cont[0].get('src')
  else:
    return None

In [58]:
def getGRlang(content, isbn):
  print('GETTING LANGUAGE ...')
  acct = 2
  cont = content.find_all('div', itemprop = 'inLanguage')
  while len(cont) == 0 and acct != 0:
    content = getContent(isbn)
    cont = content.find_all('div', itemprop = 'inLanguage')
    acct = acct - 1
  if len(cont) != 0:
    return cont[0].text
  else:
    return None

In [59]:
def getAlsoEnjoy(content, isbn):
  print('GETTING ALSO ENJOYED BOOKS ...')
  acct = 2
  cont = content.find_all('li', class_ = 'cover')
  alsoEnjoy_list = []
  while len(cont) == 0 and acct != 0:
    content = getContent(isbn)
    cont = content.find_all('li', class_ = 'cover')
    acct = acct - 1
  if len(cont) != 0:
    for book in cont:
      title = book.find('img').get('alt')
      thumbnail = book.find('img').get('src')
      alsoEnjoy_list.append([title, thumbnail])
    return alsoEnjoy_list
  else:
    return None

In [60]:
def getMoreData(dfBooks):
  current_batch_data = dfBooks.loc[dfBooks.isbn_13.notna()]
  current_batch_data = current_batch_data.loc[current_batch_data.status == -1]

  for id in current_batch_data.id:
    global current_n
    current_n = current_n + 1
    try:
      isbn = dfBooks.isbn_13[dfBooks.id == id].tolist()[0]
      index = dfBooks.isbn_13[dfBooks.id == id].index.tolist()[0]
      ### PROGRESS ###
      clear_output(wait=True)
      global batch_title
      print('dfBooks at batch ... ', batch_title)
      print('CURRENTLY ', '[', current_n,'/', save_every_n, ']')
      print('NOW AT INDEX ... ',index,'/',len(dfBooks))
      print(len(jsonError),' JSON ERROR')
      ################
      if not404(isbn) and GRhasBook(isbn) and GRhasRatings(isbn):
        content = getContent(isbn)
        genre = getGenre(content, isbn)
        text_reviews = getTextReviews(content, isbn)

        if content is None:
          content = getContent(isbn)

        desc = getGRdesc(content, isbn)
        alsoEnjoy = getAlsoEnjoy(content, isbn)
        thumbnail = getGRthumbnail(content, isbn)

        if dfBooks.at[index, 'language'] is None or dfBooks.at[index, 'language'] is np.NaN:
          lang = getGRlang(content, isbn)
          dfBooks.at[index, 'language'] = lang

        dfBooks.at[index, 'genre'] = genre
        dfBooks.at[index, 'text_reviews'] = text_reviews
        dfBooks.at[index, 'goodreads_desc'] = desc
        dfBooks.at[index, 'also_enjoy'] = alsoEnjoy
        dfBooks.at[index, 'status'] = 1

      else:
        dfBooks.at[index, 'status'] = 0

    except KeyError as v: # JSON ERROR
      print('WHAT')
      jsonError.append(isbn)
      dfErr = {'Barcode':isbn}
      books_isbn = books_isbn.append(dfErr, ignore_index=True)
    if current_n == save_every_n:
      batch_title = batch_title + 1
      path = '/content/drive/MyDrive/Chaloklum bookshop dataset/v_' + dataset_version + '/books_dataset_w_review_' + dataset_version + '_batch_' + str(batch_title) + file_type
      dfBooks.to_csv(path, index=False)

      dfJSONerror = pd.DataFrame(jsonError)
      dfJSONerror.to_csv('/content/drive/MyDrive/Chaloklum bookshop dataset/v_' + dataset_version + '/json_error_' + dataset_version + file_type, index=False)

      current_n = 0


In [61]:
def isDuplicate(isbn):
  if isbn in dfBooks['isbn_book'].values:
    return True
  else:
    return False

# Load requirement data here

In [ ]:
### END OF FUNCTIONS ###

In [62]:
dataset_version = '2.4.0'

In [64]:
### READ FILE HERE ###
books_isbn = pd.read_csv(
    '/content/drive/MyDrive/Chaloklum bookshop dataset/books' + dataset_version + '.txt',  
    names=['Barcode'])

In [66]:
### Open last version of books dataset ###
dfBooks = pd.read_csv(
    '/content/drive/MyDrive/Chaloklum bookshop dataset/v_2.3.5/books_dataset_w_review_2.3.5_final.csv')

# Call functions here

In [80]:
#books_isbn

In [ ]:
### CALL FUNCTIONS HERE ###

In [67]:
newBooks(books_isbn)

18

' no book found: '

'8935036679840'

In [70]:
retryAPIout(apiRestBooks)

In [71]:
addGRbooks(noBooksList)

['CHECKING >>> ', '8935036679840']

['NO INFO BOOKS: ', 11, ' LEFT']

['404 REQUEST: ', 0]

In [72]:
save_every_n = 100
current_n = 0
batch_title = 0
file_type = '.csv'

In [73]:
getMoreData(dfBooks)

dfBooks at batch ...  0
CURRENTLY  [ 15 / 100 ]
NOW AT INDEX ...  3929 / 3937
0  JSON ERROR
GETTING CONTENT >>>  9781848875197
GETTING TEXT REVIEWS >>>  30  REVIEWS
GETTING DESCRIPTION ...
GETTING ALSO ENJOYED BOOKS ...
GETTING THUMBNAIL ...


In [75]:
path = '/content/drive/MyDrive/Chaloklum bookshop dataset/v_' + dataset_version + '/books_dataset_w_review_' + dataset_version + '_final' + file_type
dfBooks.to_csv(path, index=False)

In [74]:
dfBooks

,id,isbn_10,isbn_13,isbn_other,isbn_book,authors,title,subtitle,publisher,published_date,...,rating_avg,#reviews,#ratings,#text_reviews,thumbnail,genre,goodreads_desc,text_reviews,also_enjoy,status
0,1,NaN,NaN,NaN,9788020609564,['Jiří Šolc'],Útěky a návraty Bohumila Laušmana,osud českého politika,NaN,2008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0
1,2,NaN,NaN,NaN,9789637253089,NaN,A nők tartják az égbolt felét,egy rendkívüli asszony rendkívüli élettörténete,NaN,2005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0
2,3,068808835X,9780688088354,NaN,068808835X,['Anne Edwards'],Wallis,The Novel,William Morrow & Company,1991,...,3.74,312.0,151.0,22.0,http://books.google.com/books/content?id=wOyMH...,"['Historical', 'Historical Fiction', 'Fiction'...",\nBorn as a poor relation to the wealthy Warfi...,"[[5.0, ""NetGalley provided me this book file i...","[['The Glorious Guinness Girls', 'https://i.gr...",1.0
3,4,068911611X,9780689116117,NaN,068911611X,['Bob Greene'],"Cheeseburgers, the Best of Bob Greene",NaN,Scribner,1985,...,4.11,165.0,93.0,9.0,http://books.google.com/books/content?id=3ssiA...,"['Nonfiction', 'Writing', 'Journalism']","\nAmusing, poignant, and often surprising, Che...","[[4.0, ""I don&apost remember all his essays, b...",[['Veeck As In Wreck: The Autobiography of Bil...,1.0
4,5,NaN,NaN,NaN,9780316914454,['Andrew Davies'],"We, the Nation",The Conservative Party and the Pursuit of Power,NaN,1995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3932,3933,None,8710425001033,NaN,8710425001033,[Barbara Pierce],Troostende armen by Barbara Pierce,None,None,None,...,3.52,187,52,4,None,NaN,NaN,NaN,NaN,NaN
3933,3934,9604695789,9789604695782,NaN,9789604695782,"[Agatha Christie, Λουκάς Λοράνδος]","Ποιος σκότωσε τον Ακρόυντ (Hercule Poirot, #4)...",None,None,None,...,4.26,411472,222766,17339,None,NaN,NaN,NaN,NaN,NaN
3934,3935,None,9789604696079,NaN,9789604696079,"[Agatha Christie, Άννα Παπαδημητρίου]","Ρετρό στην ομίχλη (Miss Marple, #13) by Agatha...",None,None,None,...,3.97,70692,38677,2313,None,NaN,NaN,NaN,NaN,NaN
3935,3936,None,9789604695799,NaN,9789604695799,"[Agatha Christie, Αχιλλέας Κυριαζής]",Ο σατανάς με την πλερέζα by Agatha Christie,None,None,None,...,3.90,68117,35194,2066,None,NaN,NaN,NaN,NaN,NaN


In [76]:
dfBooks_reorder = dfBooks[['id', 'title', 'subtitle', 'authors', 'isbn_10', 'isbn_13', 'isbn_other', 'isbn_book', 'language', 'google_desc', 'categories', 'goodreads_desc', 'publisher', 'published_date', 'page_count', 'rating_avg', '#reviews', '#ratings', '#text_reviews', 'thumbnail', 'genre', 'text_reviews', 'also_enjoy', 'status']]

In [77]:
path_reorder = '/content/drive/MyDrive/Chaloklum bookshop dataset/v_' + dataset_version + '/books_dataset_w_review_' + dataset_version + '_reordered' + file_type
dfBooks_reorder.to_csv(path_reorder, index=False)

In [78]:
dfBooks_for_finder = dfBooks_reorder.drop(columns=['text_reviews', 'thumbnail', 'also_enjoy', 'status', '#reviews',	'#ratings',	'#text_reviews', 'isbn_10', 'isbn_13', 'isbn_other'])

In [79]:
path_for_finder = '/content/drive/MyDrive/Chaloklum bookshop dataset/v_' + dataset_version + '/books_dataset_' + dataset_version + '_for_finder' + file_type
dfBooks_for_finder.to_csv(path_for_finder, index=False)